## TRying out word embeddings

### fastai https://github.com/fastai/fastai extracting sentiment(?)

### sbert  sentence transformers https://github.com/UKPLab/sentence-transformers summarize text into one sentence - > extract keywords?

### spacy 
https://github.com/explosion/spaCy 
- text classification (assigning categories of labels to text)
- similarity (comparing words to each other)
- word vectors  - tokenizes sentenses by words
- entity recognition (real object etc)


### Robert Lange Tranformers (Hugging Face)
https://huggingface.co/roberta-large-mnli?text=The+dog+was+lost.+Nobody+lost+any+animal
- assigning categories from the list

### DistilBert (Hugging Face)

### Fastext for word embedding and classification

In [70]:
x = [“Nothing suits me like suit”] # Extract ELMo features 
embeddings = elmo(x, signature=”default”, as_dict=True)[“elmo”] 
embeddings.shape

SyntaxError: invalid character in identifier (1937367061.py, line 1)

In [3]:
import sys
import pandas as pd
import itertools

In [4]:
# Add src folder to the path
sys.path.append('../src/')
from data.preprocess_keywords import make_cleaned_keywords_df
from data.make_datasets import get_data
# Specify wanted time range
start_date = '2019-01-01'
end_date = '2022-01-01'
# Path to data
data_file = '../data/raw/CMS_2010_to_June_2022_ENGLISH.json'
# Load and extract data within time range
df_subset = get_data(data_file, start_date, end_date)
# Cleans keywords and saves data as a dataframe
make_cleaned_keywords_df(df_subset, start_date, end_date)

# Load data
filepath = '../data/interim/clean_keywords_' + start_date + '_' + end_date + '.json'
df = pd.read_json(filepath, orient ='split', compression = 'infer')

Getting data...
Loading data DONE. Number of articles is 175659
Extracting data DONE. Number of articles from 2019-01-01 to 2022-01-01 is 33829
Cleaning step 1 out of 2 DONE. Number of unique keywords went from 32682 to 30228
Cleaning step 2 out of 2 DONE. Number of unique keywords went from 30228 to 27989
Finished. Data stored in ../data/interim/clean_keywords_2019-01-01_2022-01-01.json


In [19]:
#encode list of words
lang_lst= (df['keywordStringsCleanAfterFuzz'][1])
encoded_lst = [ [e.encode('UTF-8') for e in lang_lst]] 
print(encoded_lst)

[[b'english channel', b'migration', b'boats', b'illegal immigration']]


##FASTTEXT

In [21]:
lang_str = ', '.join(lang_lst)
print(lang_str)
lang_str.encode('UTF-8')

english channel, migration, boats, illegal immigration


b'english channel, migration, boats, illegal immigration'

In [ ]:
model = fasttext.train_unsupervised('text-utf8.txt', model='skipgram')

In [16]:
list(itertools.chain(*list(df['keywordStringsCleanAfterFuzz'][:5])))

['nasa',
 'osiris-rex',
 'bennu',
 'asteroid',
 'english channel',
 'migration',
 'boats',
 'illegal immigration',
 'brazil',
 'jair bolsonaro',
 'chicago economics',
 'hamilton mourao',
 'paulo guedes',
 'japan',
 'tokyo',
 'harajuku',
 'attack',
 'asia',
 'bangladesh',
 'elections',
 'kamal hossain',
 'sheikh hasina',
 'awami league',
 'khaleda zia']

In [8]:
#make flat keyword list
flat_keywords = list(itertools.chain(*list(df['keywordStringsCleanAfterFuzz'])))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33829 entries, 0 to 33828
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   id                            33829 non-null  int64 
 1   lastModifiedDate              33829 non-null  object
 2   keywordStrings                33829 non-null  object
 3   keywordStringsCleanAfterFuzz  33829 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


NameError: name 'df_clean' is not defined

In [11]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [12]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Apple Apple PROPN NNP nsubj Xxxxx True False
is be AUX VBZ aux xx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. U.K. PROPN NNP dobj X.X. False False
startup startup NOUN NN dep xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False


In [48]:
(flat_keywords[5])

'migration'

In [49]:
import spacy

nlp = spacy.load("en_core_web_lg")
tokens = nlp(flat_keywords[5])

for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)


migration True 52.682186 False


In [50]:
import spacy
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp("This is a sentence.")
print([(w.text, w.pos_) for w in doc])

[('This', 'PRON'), ('is', 'AUX'), ('a', 'DET'), ('sentence', 'NOUN'), ('.', 'PUNCT')]


## Tranformers From Huggin Face : Roberta Large 
https://huggingface.co/roberta-large-mnli?text=The+dog+was+lost.+Nobody+lost+any+animal

In [1]:
from transformers import pipeline
classifier = pipeline('zero-shot-classification', model='roberta-large-mnli')

2023-03-27 17:11:58.877968: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 17:11:59.082704: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 17:11:59.082728: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-27 17:12:00.132584: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

Downloading:   0%|          | 0.00/688 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [1]:

(df['keywordStringsCleanAfterFuzz'][55])

NameError: name 'df' is not defined

In [29]:
#sequence_to_classify = "German chancellor"
line_keywords = str(df['keywordStringsCleanAfterFuzz'][561])
#candidate_labels = ['politics','travel', 'cooking', 'dancing']
dw_categories =  ['History', 'Business', 'Conflicts', 'Politics', 'Culture',
       'Society', 'Education', 'Digital World', 'Sports', 'Crime',
       'Lifestyle', 'Religion', 'Human Rights', 'Science', 'Music',
       'Law and Justice', 'Terrorism', 'Nature and Environment', 'Film',
       'Travel', 'Media', 'Technology', 'Soccer', 'Learning German',
       'Catastrophe', 'Cars and Transportation', 'Health', 'Literature',
       'Arts', 'Architecture', 'Dance', 'Design', 'Theater', 'Migration',
       'Freedom of Speech', 'Trade', 'Rule of Law', 'Press Freedom',
       'Offbeat', 'Equality', 'Climate', 'Corruption', 'Diversity',
       'Innovation', 'Globalization', 'Food Security']
classifier(line_keywords, dw_categories)

{'sequence': "['mischael modrikamen', 'steve bannon', 'donald trump', 'populism', 'nationalism', 'european elections', 'matteo salvini', 'afd', 'conflict zone', 'tim sebastian', '#dwzone']",
 'labels': ['Conflicts',
  'Offbeat',
  'Politics',
  'Media',
  'Music',
  'Freedom of Speech',
  'Diversity',
  'Travel',
  'Learning German',
  'Society',
  'Film',
  'Dance',
  'Migration',
  'History',
  'Digital World',
  'Culture',
  'Literature',
  'Catastrophe',
  'Technology',
  'Law and Justice',
  'Arts',
  'Innovation',
  'Rule of Law',
  'Health',
  'Terrorism',
  'Theater',
  'Human Rights',
  'Sports',
  'Corruption',
  'Globalization',
  'Crime',
  'Soccer',
  'Climate',
  'Equality',
  'Religion',
  'Lifestyle',
  'Press Freedom',
  'Trade',
  'Design',
  'Education',
  'Cars and Transportation',
  'Architecture',
  'Nature and Environment',
  'Business',
  'Science',
  'Food Security'],
 'scores': [0.11758334189653397,
  0.10028944164514542,
  0.09374994039535522,
  0.08295890688

In [30]:
#from transformers import pipeline

dbert_classif=pipeline('zero-shot-classification', model='distilbert-base-uncased-distilled-squad')

Downloading:   0%|          | 0.00/451 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased-distilled-squad were not used when initializing DistilBertForSequenceClassification: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-distilled-squad and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df['keywordStringsCleanAfterFuzz'].astype(str))
labels = dw_categories
features.shape

(33829, 129640)

In [52]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

#category_id_df = df_clean['cleanFocusCategory'].drop_duplicates().sort_values()
model = LinearSVC()
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0,stratify=labels)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#from sklearn.metrics import confusion_matrix
#conf_mat = confusion_matrix(y_test, y_pred)
#fig, ax = plt.subplots(figsize=(20,20))
#sns.heatmap(conf_mat, annot=True, xticklabels=category_id_df.values, yticklabels=category_id_df.values)
#plt.ylabel('Actual')
#plt.xlabel('Predicted')
#plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [33829, 46, 33829]

In [53]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

NameError: name 'y_test' is not defined

In [54]:
df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            33819, 33820, 33821, 33822, 33823, 33824, 33825, 33826, 33827,
            33828],
           dtype='int64', length=33829)